<div style="background-color:rgba(128, 0, 128, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">MNIST: CNN</h1>
</div>

Problem Type: Multi-classification

## Evaluation Metric

In [1]:
# Black formatter https://black.readthedocs.io/en/stable/

! pip install nb-black > /dev/null

%load_ext lab_black

<div style="background-color:rgba(128, 0, 128, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Configuration</h1>
</div>

In [2]:
TARGET = "label"

<div style="background-color:rgba(128, 0, 128, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Import Libraries</h1>
</div>

A best practise is to include all libraries here.  However, I will put a few imports farther down where they are first used so beginners can learn with an "as needed" approach.

In [3]:
import os
import time
from pathlib import Path

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score

<div style="background-color:rgba(128, 0, 128, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Library</h1>
</div>

Creating a few functions that we will reuse in each project.

In [4]:
def read_data(path):
    data_dir = Path(path)

    train = pd.read_csv(data_dir / "train.csv")
    test = pd.read_csv(data_dir / "test.csv")
    submission_df = pd.read_csv(data_dir / "sample_submission.csv")

    print(f"train data: Rows={train.shape[0]}, Columns={train.shape[1]}")
    print(f"test data : Rows={test.shape[0]}, Columns={test.shape[1]}")
    return train, test, submission_df

In [5]:
def create_submission(model_name, target, preds, is_log_target=False):
    if is_log_target:
        #         preds = np.exp(preds)
        preds = np.expm1(preds)

    sample_submission[target] = preds
    if len(model_name) > 0:
        sample_submission.to_csv(f"submission_{model_name}.csv", index=False)
    else:
        sample_submission.to_csv(f"submission.csv", index=False)

    return sample_submission[:5]

In [6]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, f1_score


def show_scores(gt, yhat):
    f1 = f1_score(gt, yhat, average="macro")
    mse = mean_squared_error(gt, yhat)
    mae = mean_absolute_error(gt, yhat)
    rmse = np.sqrt(mse)
    #     r2_square = r2_score(gt, yhat)

    print(f"f1: {f1:.4f}")

<div style="background-color:rgba(128, 0, 128, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Load Train/Test Data</h1>
</div>

- train.csv - Data used to build our machine learning model
- test.csv - Data used to build our machine learning model. Does not contain the target variable
- sample_submission.csv - A file in the proper format to submit test predictions

In [7]:
train, test, sample_submission = read_data("../input/digit-recognizer")

train data: Rows=42000, Columns=785
test data : Rows=28000, Columns=784


In [8]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
FEATURES = [
    col
    for col in train.columns
    if col
    not in [
        "Id",
        TARGET,
        #         TARGET_ENC,
    ]
]

In [10]:
y = train[TARGET]
X = train[FEATURES].copy()

X_test = test[FEATURES].copy()

In [11]:
# Normalize the data
X = X / 255.0
X_test = X_test / 255.0

In [12]:
X = X.values.reshape(-1, 28, 28, 1)
X_test = X_test.values.reshape(-1, 28, 28, 1)

In [13]:
# X[:3]

In [14]:
from keras.utils.np_utils import to_categorical

y = to_categorical(y)

In [15]:
y[:5]

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [16]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X,
    y,
    test_size=0.1,
    random_state=42,
)
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((37800, 28, 28, 1), (37800, 10), (4200, 28, 28, 1), (4200, 10))

<div style="background-color:rgba(128, 0, 128, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Create Models</h1>
</div>

In [17]:
### import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


def get_tf_model01(x_shape):
    print(f"x_shape=")
    #     input_shape = (height, width, channels)
    inputs = keras.Input(shape=(28, 28, 1))

    x = keras.layers.Conv2D(
        32,
        kernel_size=3,
        activation="relu",
        padding="same",
        input_shape=(28, 28, 1),
        kernel_initializer="he_normal",
    )(inputs)

    x = keras.layers.Conv2D(
        64,
        kernel_size=3,
        activation="relu",
        padding="same",
        kernel_initializer="he_normal",
    )(x)

    x = keras.layers.BatchNormalization()(x)

    #     x = keras.layers.MaxPooling2d((2, 2))(x)
    #     x = keras.layers.Conv2D(
    #         128, activation="relu", padding="same", kernel_initializer="he_normal"
    #     )(x)
    x = keras.layers.Flatten()(x)
    #     x = keras.layers.Dense(128, activation="relu")(x)

    x = keras.layers.Dense(64, activation="relu")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dense(32, activation="relu")(x)
    x = keras.layers.BatchNormalization()(x)
    #     x = keras.layers.Dropout(0.1)(x)

    outputs = keras.layers.Dense(10, activation="softmax")(x)

    model = keras.Model(inputs, outputs)

    model.compile(
        loss="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"]
    )

    #     model.compile(
    #         optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    #         loss="mean_absolute_error",
    #     )

    return model

In [18]:
X_train.shape[1]

28

In [19]:
X.shape[0]

42000

In [20]:
model = get_tf_model01(x_shape=(28, 28))

model.summary()

x_shape=


2022-07-16 00:22:28.593030: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-16 00:22:28.705470: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-16 00:22:28.706241: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-16 00:22:28.707875: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 64)        256       
_________________________________________________________________
flatten (Flatten)            (None, 50176)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                3211328   
_________________________________________________________________
batch_normalization_1 (Batch (None, 64)                256   

In [21]:
%%time

history = model.fit(
    X_train,
    y_train,
    batch_size=64,
    epochs=10,
    # Suppress logging.
    verbose=1,
    # Calculate validation results on 20% of the training data.
    #     validation_split = 0.2
)

2022-07-16 00:22:31.961774: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10


2022-07-16 00:22:33.991551: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


591/591 [==============================] - 12s 6ms/step - loss: 0.1578 - accuracy: 0.9584
Epoch 2/10
591/591 [==============================] - 4s 7ms/step - loss: 0.0447 - accuracy: 0.9862
Epoch 3/10
591/591 [==============================] - 4s 6ms/step - loss: 0.0234 - accuracy: 0.9927
Epoch 4/10
591/591 [==============================] - 4s 6ms/step - loss: 0.0151 - accuracy: 0.9952
Epoch 5/10
591/591 [==============================] - 4s 7ms/step - loss: 0.0095 - accuracy: 0.9971
Epoch 6/10
591/591 [==============================] - 4s 6ms/step - loss: 0.0059 - accuracy: 0.9981
Epoch 7/10
591/591 [==============================] - 4s 6ms/step - loss: 0.0055 - accuracy: 0.9983
Epoch 8/10
591/591 [==============================] - 4s 7ms/step - loss: 0.0041 - accuracy: 0.9987
Epoch 9/10
591/591 [==============================] - 4s 7ms/step - loss: 0.0030 - accuracy: 0.9991
Epoch 10/10
591/591 [==============================] - 4s 6ms/step - loss: 0.0026 - accuracy: 0.9992
CPU times

## Predict and Score

In [22]:
valid_preds = model.predict(X_valid)
valid_preds[:5]

array([[3.0732251e-06, 1.0587127e-06, 5.8235810e-04, 8.8470509e-10,
        1.2973078e-06, 3.4643719e-07, 1.3323972e-06, 9.1176481e-08,
        9.9941015e-01, 3.4648610e-07],
       [5.7223730e-08, 9.9995899e-01, 2.7493572e-05, 1.3918042e-09,
        2.2313044e-08, 7.6055153e-09, 9.2684587e-07, 1.2000393e-05,
        5.2321769e-07, 7.1589201e-09],
       [5.1604253e-08, 1.8434207e-08, 4.3133892e-09, 6.1713314e-07,
        1.5253330e-06, 3.1509689e-08, 7.7818434e-08, 8.5403443e-08,
        1.3894257e-07, 9.9999750e-01],
       [9.1883848e-08, 1.8713788e-08, 6.5768049e-09, 7.2114758e-06,
        1.7932705e-07, 1.2353209e-07, 1.5527924e-08, 2.0043764e-05,
        1.7356815e-06, 9.9997056e-01],
       [2.7263537e-08, 1.4967165e-08, 3.5183443e-09, 3.8314644e-07,
        1.5101374e-07, 3.9330450e-09, 1.7274605e-09, 1.3018780e-09,
        9.9999940e-01, 1.7633880e-08]], dtype=float32)

In [23]:
valid_preds = np.argmax(valid_preds, axis=1)

In [24]:
# show_scores(y_valid, valid_preds)



In [25]:
preds = model.predict(X_test)
preds[:5]

array([[8.21539711e-12, 1.36976444e-10, 1.00000000e+00, 1.03808585e-11,
        3.77372550e-10, 1.46870138e-11, 1.12966622e-10, 2.59443400e-09,
        8.15164380e-10, 1.65625222e-10],
       [9.99999881e-01, 9.26401622e-10, 4.86625495e-08, 1.08789010e-07,
        1.83756264e-08, 5.13247611e-10, 9.17959309e-09, 8.36763725e-09,
        4.41490533e-09, 2.00085670e-10],
       [1.09288081e-08, 3.21950964e-08, 5.23863673e-05, 3.82275033e-07,
        1.30585777e-05, 3.59982164e-06, 2.11486508e-07, 2.36613346e-06,
        6.83198095e-07, 9.99927282e-01],
       [5.38967688e-05, 7.47786331e-08, 7.80241010e-07, 9.58836477e-09,
        6.70143254e-06, 1.49296469e-08, 8.55520227e-07, 7.39828192e-06,
        1.21802384e-06, 9.99929070e-01],
       [7.54491174e-12, 4.26977377e-11, 1.91947919e-07, 9.99999762e-01,
        1.59255553e-10, 4.02456124e-11, 2.26388810e-10, 3.65143638e-10,
        3.10518367e-09, 5.32157859e-11]], dtype=float32)

In [26]:
preds = np.argmax(preds, axis=1)

## Submission

In [27]:
# Different TARGET name used in test.csv
ss = create_submission("", "Label", preds)
ss

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
